In [1]:
'''
Author: ChenHJ
Date: 2022-04-16 23:56:17
LastEditors: ChenHJ
LastEditTime: 2022-04-16 23:56:17
FilePath: /chenhj/0302code/cal_SAM_regress.py
Aim: 
Mission: 
'''
# %%
import numpy as np
import xarray as xr
import os
import re
from cdo import Cdo
import shutil
import sys

sys.path.append("/home/ys17-23/chenhj/self_def/")
import plot as sepl
import cal as ca
import pandas as pd
from importlib import reload

import metpy.calc as mpcalc
import metpy.xarray as mpxr
import metpy.constants as constants
import geocat.comp
from windspharm.xarray import VectorWind

reload(sepl)

# sd.path.append("/home/ys17-23/chenhj/1201code/self_def.py")

cdo = Cdo()

# for plot
import proplot as pplt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter
from cartopy.mpl.ticker import LatitudeFormatter
from cartopy.util import add_cyclic_point
from matplotlib.ticker import MultipleLocator
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import t
from scipy import signal
from eofs.multivariate.standard import MultivariateEof
from eofs.standard import Eof


def patches(ax, x0, y0, width, height, proj):
    from matplotlib.patches import Rectangle

    rect = Rectangle(
        (x0, y0), width, height, fc="none", ec="grey7", linewidth=0.8, zorder=1.1, transform=proj, linestyle="--",
    )
    ax.add_patch(rect)

In [2]:
#   read the ERA5 reanalysis data and calculate the SAM and IWF index
fhgtERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/hgt_mon_r144x72_195001-201412.nc"
)
hgtERA5 = fhgtERA5["z"]


fuERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/uwind_mon_r144x72_195001-201412.nc"
)
uERA5 = fuERA5["u"]

fvERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/vwind_mon_r144x72_195001-201412.nc"
)
vERA5 = fvERA5["v"]

fspERA5 = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/sp_mon_r144x72_195001-201412.nc"
)
spERA5 = fspERA5["sp"]

fqERA5 = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/q_mon_r144x72_195001-201412.nc")
qERA5 = fqERA5["q"]

In [3]:
#   read the precipitation data
fpreCRU = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/cru_ts4.01_r144x72_195001-201412.nc"
)
preCRU = fpreCRU["pre"]

# GPCP data just have 1979-2014 year
fpreGPCP = xr.open_dataset(
    "/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/obs/GPCP_r144x72_197901-201412.nc"
)
preGPCP = fpreGPCP["precip"]

In [4]:
hgtERA5_ver_JJA = ca.p_time(hgtERA5, 6, 8, True).loc[:, 100.0:, :, :]
uERA5_ver_JJA = ca.p_time(uERA5, 6, 8, True).loc[:, 100.0:, :, :]
vERA5_ver_JJA = ca.p_time(vERA5, 6, 8, True).loc[:, 100.0:, :, :]
qERA5_ver_JJA = ca.p_time(qERA5, 6, 9, True).loc[:, 100.0:, :, :]
preCRU_JJA = ca.p_time(preCRU, 6, 8, True) / 30.67
spERA5_JJA = ca.p_time(spERA5, 6, 8, True)

hgtERA5_ver_JJA = ca.detrend_dim(hgtERA5_ver_JJA, "time", deg=1, demean=False)
uERA5_ver_JJA = ca.detrend_dim(uERA5_ver_JJA, "time", deg=1, demean=False)
vERA5_ver_JJA = ca.detrend_dim(vERA5_ver_JJA, "time", deg=1, demean=False)
qERA5_ver_JJA = ca.detrend_dim(qERA5_ver_JJA, "time", deg=1, demean=False)
preCRU_JJA = ca.detrend_dim(preCRU_JJA, "time", deg=1, demean=False)
spERA5_JJA = ca.detrend_dim(spERA5_JJA, "time", deg=1, demean=False)

preGPCP_JJA = ca.p_time(preGPCP, 6, 8, True)
preGPCP_JJA = ca.detrend_dim(preGPCP_JJA, "time", deg=1, demean=False)

In [5]:
#   calculate the SAM index in ERA5
ERA5_SAM_index = ca.SAM(vERA5_ver_JJA)
ERA5_SAM_index = ca.detrend_dim(ERA5_SAM_index, "time", deg=1, demean=False)

In [6]:
# read the SAM index of historical run and ssp585
fhis_SAM_index = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/his_SAM_index_1950-2014.nc")
his_SAM_index = fhis_SAM_index["SAM"]

fssp585_SAM_index = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_SAM_index_2015-2099.nc")
ssp585_SAM_index = fssp585_SAM_index["SAM"]

In [7]:
# read the SAM index and precipitation in historical run and ssp585
fhis_SAM_index = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/his_SAM_index_1950-2014.nc")
his_SAM_index = fhis_SAM_index["SAM"]

fssp585_SAM_index = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/ssp585_SAM_index_2015-2099.nc")
ssp585_SAM_index = fssp585_SAM_index["SAM"]

fprehis_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/historical/tmp_var/JJA/detrend/pr_historical_r144x72_195001-201412.nc")
prehis_JJA = fprehis_JJA["pr"]

fpressp585_JJA = xr.open_dataset("/home/ys17-23/Extension/personal-data/chenhj/SAM_EAM_data/CMIP6/ssp585/tmp_var/JJA/detrend/pr_ssp585_r144x72_201501-209912.nc")
pressp585_JJA = fpressp585_JJA["pr"]

In [8]:
#   pick up the precipitation in India area
lat = preCRU_JJA.coords["lat"]
lon = preCRU_JJA.coords["lon"]
lat_India_range = lat[(lat >= 8.0) & (lat <= 28.0)]
lon_India_range = lon[(lon >= 70.0) & (lon <= 86.0)]
preCRU_India_JJA = ca.cal_lat_weighted_mean(preCRU_JJA.sel(lat=lat_India_range, lon=lon_India_range)).mean(dim="lon", skipna=True)

lat = preGPCP_JJA.coords["lat"]
lon = preGPCP_JJA.coords["lon"]
lat_India_range = lat[(lat >= 8.0) & (lat <= 28.0)]
lon_India_range = lon[(lon >= 70.0) & (lon <= 86.0)]
preGPCP_India_JJA = ca.cal_lat_weighted_mean(preGPCP_JJA.sel(lat=lat_India_range, lon=lon_India_range)).mean(dim="lon", skipna=True)

lat = prehis_JJA.coords["lat"]
lon = prehis_JJA.coords["lon"]
lat_India_range = lat[(lat >= 8.0) & (lat <= 28.0)]
lon_India_range = lon[(lon >= 70.0) & (lon <= 86.0)]
prehis_India_JJA = ca.cal_lat_weighted_mean(prehis_JJA.sel(lat=lat_India_range, lon=lon_India_range)).mean(dim="lon", skipna=True)

lat = pressp585_JJA.coords["lat"]
lon = pressp585_JJA.coords["lon"]
lat_India_range = lat[(lat >= 8.0) & (lat <= 28.0)]
lon_India_range = lon[(lon >= 70.0) & (lon <= 86.0)]
pressp585_India_JJA = ca.cal_lat_weighted_mean(pressp585_JJA.sel(lat=lat_India_range, lon=lon_India_range)).mean(dim="lon", skipna=True)

del(lat, lon)

#   calculate the correlation coefficients between SAM and IndR
IndR_ERA5_SAM_regress = stats.linregress(preCRU_India_JJA, ERA5_SAM_index)
IndR_his_SAM_regress = ca.dim_linregress(prehis_India_JJA, his_SAM_index)

In [9]:
#   pick up the precipitation in India area
lat = preCRU_JJA.coords["lat"]
lon = preCRU_JJA.coords["lon"]
lat_India_range = lat[(lat >= 8.0) & (lat <= 28.0)]
lon_India_range = lon[(lon >= 70.0) & (lon <= 86.0)]
preCRU_India_JJA = ca.cal_lat_weighted_mean(preCRU_JJA.sel(lat=lat_India_range, lon=lon_India_range)).mean(dim="lon", skipna=True)

lat = preGPCP_JJA.coords["lat"]
lon = preGPCP_JJA.coords["lon"]
lat_India_range = lat[(lat >= 8.0) & (lat <= 28.0)]
lon_India_range = lon[(lon >= 70.0) & (lon <= 86.0)]
preGPCP_India_JJA = ca.cal_lat_weighted_mean(preGPCP_JJA.sel(lat=lat_India_range, lon=lon_India_range)).mean(dim="lon", skipna=True)

lat = prehis_JJA.coords["lat"]
lon = prehis_JJA.coords["lon"]
lat_India_range = lat[(lat >= 8.0) & (lat <= 28.0)]
lon_India_range = lon[(lon >= 70.0) & (lon <= 86.0)]
prehis_India_JJA = ca.cal_lat_weighted_mean(prehis_JJA.sel(lat=lat_India_range, lon=lon_India_range)).mean(dim="lon", skipna=True)

lat = pressp585_JJA.coords["lat"]
lon = pressp585_JJA.coords["lon"]
lat_India_range = lat[(lat >= 8.0) & (lat <= 28.0)]
lon_India_range = lon[(lon >= 70.0) & (lon <= 86.0)]
pressp585_India_JJA = ca.cal_lat_weighted_mean(pressp585_JJA.sel(lat=lat_India_range, lon=lon_India_range)).mean(dim="lon", skipna=True)

del(lat, lon)

#   calculate the correlation coefficients between SAM and IndR
IndR_ERA5_SAM_regress = stats.linregress(preCRU_India_JJA, ERA5_SAM_index)
IndR_his_SAM_regress = ca.dim_linregress(prehis_India_JJA, his_SAM_index)
IndR_ssp585_SAM_regress = ca.dim_linregress(pressp585_India_JJA, ssp585_SAM_index)

In [10]:
IndR_ERA5_SAM_regress

LinregressResult(slope=0.3386405851030982, intercept=1.9091403841419, rvalue=0.4555009731096198, pvalue=0.00013747729399749787, stderr=0.08338428824340081, intercept_stderr=0.5117961348692823)

In [11]:
IndR_his_SAM_regress[2]

<xarray.DataArray 'pr' (models: 26)>
array([ 0.39983315, -0.01454011,  0.14453293, -0.12060899,  0.15762913,
       -0.07813133, -0.10954879,  0.02364773, -0.08549423,  0.11876162,
       -0.1568878 , -0.11421499,  0.15237529,  0.09214481,  0.14730439,
        0.2577996 ,  0.3282806 ,  0.25263926, -0.24504101, -0.29941229,
        0.1605039 , -0.13103014,  0.05328003, -0.06073225, -0.2069068 ,
        0.18862002])
Coordinates:
  * models   (models) object 'ACCESS-CM2' 'BCC-CSM2-MR' ... 'UKESM1-0-LL'
Attributes:
    standard_name:  precipitation_flux
    long_name:      Precipitation
    units:          kg m-2 s-1
    comment:        includes both liquid and solid phases
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    history:        2019-11-08T08:26:36Z altered by CMOR: replaced missing va...

In [12]:
IndR_his_SAM_regress[3]

<xarray.DataArray 'pr' (models: 26)>
array([0.00096749, 0.90847877, 0.25068161, 0.33856074, 0.20982562,
       0.53615399, 0.38501562, 0.85167564, 0.4983148 , 0.3460653 ,
       0.21200436, 0.36497134, 0.22561077, 0.46536827, 0.24161353,
       0.03814391, 0.00759063, 0.04231718, 0.04914203, 0.01539421,
       0.20152654, 0.29815645, 0.67337385, 0.63081326, 0.09818498,
       0.1323911 ])
Coordinates:
  * models   (models) object 'ACCESS-CM2' 'BCC-CSM2-MR' ... 'UKESM1-0-LL'
Attributes:
    standard_name:  precipitation_flux
    long_name:      Precipitation
    units:          kg m-2 s-1
    comment:        includes both liquid and solid phases
    cell_methods:   area: time: mean
    cell_measures:  area: areacella
    history:        2019-11-08T08:26:36Z altered by CMOR: replaced missing va...

In [13]:
IndR_ssp585_SAM_regress[2]

<xarray.DataArray 'pr' (models: 26)>
array([ 0.28711296, -0.20154138,  0.14912043, -0.13682539, -0.12217848,
       -0.10834591, -0.2560287 , -0.12555856, -0.13641298,  0.0774627 ,
       -0.09868442,  0.05962853,  0.02112329, -0.06054362,  0.13991182,
        0.08088615,  0.16172163,  0.13842508, -0.06610811, -0.193305  ,
       -0.01931523, -0.04814431,  0.0272851 , -0.03349001, -0.34247676,
        0.08851987])
Coordinates:
  * models   (models) object 'ACCESS-CM2' 'BCC-CSM2-MR' ... 'UKESM1-0-LL'
Attributes:
    standard_name:  precipitation_flux
    long_name:      Precipitation
    units:          kg m-2 s-1
    comment:        includes both liquid and solid phases
    original_name:  PRECT
    cell_methods:   area: time: mean (interval: 5 minutes)
    cell_measures:  area: areacella

In [14]:
IndR_ssp585_SAM_regress[3]

<xarray.DataArray 'pr' (models: 26)>
array([0.00771716, 0.06436596, 0.17316772, 0.21178506, 0.26530886,
       0.32363324, 0.018026  , 0.25221292, 0.21317853, 0.48102409,
       0.36889232, 0.58775592, 0.84783229, 0.58202775, 0.20156119,
       0.46179224, 0.13922691, 0.20644106, 0.54775971, 0.07630235,
       0.8607203 , 0.66171222, 0.80422905, 0.76091739, 0.00133503,
       0.42046639])
Coordinates:
  * models   (models) object 'ACCESS-CM2' 'BCC-CSM2-MR' ... 'UKESM1-0-LL'
Attributes:
    standard_name:  precipitation_flux
    long_name:      Precipitation
    units:          kg m-2 s-1
    comment:        includes both liquid and solid phases
    original_name:  PRECT
    cell_methods:   area: time: mean (interval: 5 minutes)
    cell_measures:  area: areacella